In [1]:

from google.oauth2 import service_account
import pandas_gbq as gbq
import json
from pandas_gbq import to_gbq, read_gbq
import pandas as pd
import numpy as np
from pandarallel import pandarallel

In [3]:
credentials_path = '../token.json'

# Authenticate with your credentials
credentials = service_account.Credentials.from_service_account_file(
    credentials_path, scopes=['https://www.googleapis.com/auth/bigquery'])

# Set the credentials for pandas_gbq
gbq.context.credentials = credentials

In [4]:
df = read_gbq(f"""

SELECT *
FROM capstone-402105.raw_data.compustat_na_daily

""",project_id='capstone-402105', dialect='standard')

Downloading:   1%|          |

KeyboardInterrupt: 

Downloading:   1%|          |

In [11]:
df.shape[0]

1075571

In [ ]:
df_with_ratios = pd.DataFrame(df[['datadate', 'gvkey', 'fqtr', 'fyr', 'tic', 'sic']])
df_with_ratios.isna().sum()

datadate      0
gvkey         0
fqtr        346
fyr           0
tic         694
sic           0
dtype: int64

Looks like gvkey would be a better primary key than ticker since it has no nans in this dataset. We also need to inspect financial quarter

In [ ]:
df_with_ratios[df['fqtr'].isna()]

,datadate,gvkey,fqtr,fyr,tic,sic
2746,2014-03-31,1224,NaN,12,EGN1,4924
2747,2014-06-30,1224,NaN,12,EGN1,4924
12300,2014-11-30,1951,NaN,8,PRSNQ,3845
31199,2014-12-31,3424,NaN,9,BH,5812
35114,2014-12-31,3806,NaN,9,DWSN,1382
...,...,...,...,...,...,...
1050126,2013-06-30,196766,NaN,3,ALGDF,1040
1050128,2013-09-30,196766,NaN,3,ALGDF,1040
1074260,2015-06-30,317427,NaN,3,NOMD,2030
1074262,2015-09-30,317427,NaN,3,NOMD,2030


No clear pattern in financial quarters with nan values. Just use date value since there is no nans there

In [ ]:
df_with_ratios.drop(['fqtr', 'fyr'], axis=1, inplace=True)
df_with_ratios.head()

,datadate,gvkey,tic,sic
0,2000-02-29,1004,AIR,5080
1,2000-05-31,1004,AIR,5080
2,2000-08-31,1004,AIR,5080
3,2000-11-30,1004,AIR,5080
4,2001-02-28,1004,AIR,5080


In [ ]:
##Feature Engineering
df['average_asset'] = (df['atq'] + df['atq'].shift(1)) / 2
df['average_inventory'] = (df['invtq'] + df['invtq'].shift(1)) / 2
df['average_receivables'] = (df['rectq'] + df['rectq'].shift(1)) / 2
df['average_workingcapital'] = (df['wcapq'] + df['wcapq'].shift(1)) / 2
df['cogs'] = df['cogsq'] / (df['cogsy'])

In [ ]:
#Feature Engineering
df_with_ratios['current_ratio'] = df['actq'] / (df['lctq'])
df_with_ratios['quick_ratio'] = (df['actq'] - df['invtq']) / ( df['lctq']) 
df_with_ratios['cash_ratio'] = df['chechy'] / (df['lctq'] ) 
df_with_ratios['net_working_capital'] = df['actq'] - df['lctq']
df_with_ratios['debt_ratio'] = (df['dlttq'] + df['dlcq']) / ( df['atq'])
df_with_ratios['debt_to_equity_ratio'] = (df['dlttq'] + df['dlcq']) / ( df['ceqq']) 
df_with_ratios['equity_ratio'] = df['ceqq'] / (df['atq'])
df_with_ratios['cashflow_to_debt_ratio'] = df['oancfy'] / (df['dlttq'] + df['dlcq'])
df_with_ratios['net_profit_margin'] = 100 * (df['niq'] / ( df['revtq']))
df_with_ratios['return_on_assets'] = 100 * (df['niq'] / (df['atq']))
df_with_ratios['asset_turnover'] = df['revtq'] / (df['average_asset'])
df_with_ratios['inventory_turnover'] = df['cogs'] / (df['average_inventory'])
df_with_ratios['days_in_inventory'] = 365 / (df['cogs'] / ( df['average_inventory']))
df_with_ratios['receivables_turnover'] = df['revtq'] / ( df['average_inventory'])
df_with_ratios['day_sales_outstanding'] = 365 / (df['revtq'] / (df['average_inventory']))
df_with_ratios['working_capital_turnover'] = df['revtq'] /  df['average_workingcapital']
df_with_ratios['price_to_earnings'] = df['prccq'] / (df['epsfxq'])
df_with_ratios['dividend_payout_ratio'] = df['dvpspq'] / ( df['epsfxq'])
df_with_ratios['retention_ratio'] = 1 - (df['dvpspq'] / (df['epsfxq']))
df_with_ratios['gross_margin_ratio'] = 100 * ((df['revtq'] - df['cogsq']) / ( df['revtq']))
df_with_ratios['operating_profit_margin'] = 100 * ((df['revtq'] - df['cogsq']- df['xoprq']) / (df['revtq']))
df_with_ratios['ebitda_margin'] = 100 * ((df['revtq'] - df['cogsq']- df['xoprq'] + df['dpq']) / (df['revtq']))
df_with_ratios['debt_service_coverage_ratio'] = (df['revtq'] - df['cogsq']- df['xoprq']) / (df['dlttq'] + df['dlcq'])
df_with_ratios['interest_coverage_ratio'] = df['oibdpq'] / ( df['xintq'])

In [ ]:
df_with_ratios.head(5)

,datadate,gvkey,tic,sic,current_ratio,quick_ratio,cash_ratio,net_working_capital,debt_ratio,debt_to_equity_ratio,...,day_sales_outstanding,working_capital_turnover,price_to_earnings,dividend_payout_ratio,retention_ratio,gross_margin_ratio,operating_profit_margin,ebitda_margin,debt_service_coverage_ratio,interest_coverage_ratio
0,2000-02-29,1004,AIR,5080,2.981531,1.176928,-0.021609,350.202,0.273288,0.601468,...,NaN,NaN,59.375000,0.212500,0.787500,18.234795,-72.491564,-70.807951,-0.958372,4.223950
1,2000-05-31,1004,AIR,5080,3.120983,1.069792,-0.042786,347.451,0.279030,0.608989,...,531.051575,0.645246,154.166667,0.944444,0.055556,18.944015,-75.280235,-73.157425,-0.819497,2.287927
2,2000-08-31,1004,AIR,5080,3.032279,0.989191,-0.003399,341.996,0.305692,0.673592,...,513.171702,0.701345,93.750000,0.708333,0.291667,16.176118,-77.799562,-75.858047,-0.823112,2.432365
3,2000-11-30,1004,AIR,5080,2.718261,0.897575,0.009653,325.382,0.298354,0.675752,...,594.639896,0.633329,64.843750,0.531250,0.468750,18.857738,-73.583647,-71.446755,-0.674333,2.793634
4,2001-02-28,1004,AIR,5080,3.136912,1.025058,-0.002824,358.685,0.304446,0.666264,...,637.845577,0.584946,68.000000,0.425000,0.575000,20.259808,-71.119752,-68.726602,-0.619269,3.174489


In [ ]:
print(f"length of dataset: {len(df_with_ratios)}")
print(f"shape of dataset: {df_with_ratios.shape}")

length of dataset: 1075571
shape of dataset: (1075571, 28)


In [ ]:
#check number of Nans in the dataframe
nan_breakdown = pd.DataFrame()
nan_breakdown['pct_nan'] = (df_with_ratios.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown

,pct_nan
datadate,0.000000
gvkey,0.000000
tic,0.064524
sic,0.000000
current_ratio,37.248122
quick_ratio,38.371990
cash_ratio,38.470171
net_working_capital,37.207679
debt_ratio,28.442846
debt_to_equity_ratio,28.606852


At this point our dataset should have columns with at most 20-30% nan values. we shall now inpute them using sector median. 
Let us first try Methodology 1: 
1. map gvkey to sector
2. Eg. if company X in sector A on date 2000-01-01 has a missing value for total income, we inpute it using the median of sector A in 2000-01-01. 

This assumes that on a particular date, a companies fundamentals can be represented by its sector median on that date. 
Sector will be represented by SIC code. (I used all 4 digits for now)

First let us convert datadate to year and quarter representation

In [ ]:
#convert datadate to year and quater
df_with_ratios['datadate'] = pd.to_datetime(df_with_ratios['datadate'])
df_with_ratios['year'] = df_with_ratios['datadate'].dt.year
df_with_ratios['quarter'] = df_with_ratios['datadate'].dt.quarter
df_with_ratios.head()

,datadate,gvkey,tic,sic,current_ratio,quick_ratio,cash_ratio,net_working_capital,debt_ratio,debt_to_equity_ratio,...,price_to_earnings,dividend_payout_ratio,retention_ratio,gross_margin_ratio,operating_profit_margin,ebitda_margin,debt_service_coverage_ratio,interest_coverage_ratio,year,quarter
0,2000-02-29,1004,AIR,5080,2.981531,1.176928,-0.021609,350.202,0.273288,0.601468,...,59.375000,0.212500,0.787500,18.234795,-72.491564,-70.807951,-0.958372,4.223950,2000,1
1,2000-05-31,1004,AIR,5080,3.120983,1.069792,-0.042786,347.451,0.279030,0.608989,...,154.166667,0.944444,0.055556,18.944015,-75.280235,-73.157425,-0.819497,2.287927,2000,2
2,2000-08-31,1004,AIR,5080,3.032279,0.989191,-0.003399,341.996,0.305692,0.673592,...,93.750000,0.708333,0.291667,16.176118,-77.799562,-75.858047,-0.823112,2.432365,2000,3
3,2000-11-30,1004,AIR,5080,2.718261,0.897575,0.009653,325.382,0.298354,0.675752,...,64.843750,0.531250,0.468750,18.857738,-73.583647,-71.446755,-0.674333,2.793634,2000,4
4,2001-02-28,1004,AIR,5080,3.136912,1.025058,-0.002824,358.685,0.304446,0.666264,...,68.000000,0.425000,0.575000,20.259808,-71.119752,-68.726602,-0.619269,3.174489,2001,1


In [ ]:
#drop duplicates of rows 
df_drop_duplicates = df_with_ratios.drop_duplicates(subset=['gvkey', 'year', 'quarter'])
len(df_with_ratios) - len(df_drop_duplicates)

1402

We then drop rows where there are duplicate year, quarters and gvkey, since the original dataset is monthly data. We only drop 1402 rows, so not significant. 

In [ ]:
df_with_ratios = df_drop_duplicates

Now we find all the gvkeys where there is no data at all and drop them from our dataframe. From this operation we remove 6813 gvkeys from a total of 

In [ ]:
df_drop_gvkey = df_with_ratios.drop(['sic',  'tic', 'year', 'quarter', 'datadate'], axis=1)
df_drop_gvkey = df_drop_gvkey.groupby('gvkey').apply(lambda group: group.iloc[:, 1:].isna().all().all())
to_drop = df_drop_gvkey[df_drop_gvkey == True]
gvkeys_to_drop = list(to_drop.index)

In [ ]:
original_number_of_gvkeys = len(df_with_ratios['gvkey'].unique())
print(f"Dropped {len(gvkeys_to_drop) * 100 / original_number_of_gvkeys}% of stocks")

Dropped 22.53381394887397% of stocks


In [ ]:
df_with_ratios_dropped_gvkey = df_with_ratios[~df_with_ratios['gvkey'].isin(gvkeys_to_drop)]
print(f"number of unique gvkeys now: {len(df_with_ratios_dropped_gvkey['gvkey'].unique())}")
print(f"% of total rows lost: {(len(df_with_ratios) - len(df_with_ratios_dropped_gvkey)) * 100 / len(df_with_ratios)  }%")

number of unique gvkeys now: 23425
% of total rows lost: 17.750651899282143%


In [ ]:
nan_breakdown = pd.DataFrame()
nan_breakdown['pct_nan'] = (df_with_ratios_dropped_gvkey.isna().sum() / len(df_with_ratios_dropped_gvkey)) * 100
nan_breakdown

,pct_nan
datadate,0.000000
gvkey,0.000000
tic,0.027957
sic,0.000000
current_ratio,23.707834
quick_ratio,25.075580
cash_ratio,25.167941
net_working_capital,23.658937
debt_ratio,12.990197
debt_to_equity_ratio,13.190650


We can also see that the % of nans has decreased significantly across columns after dropping these gvkeys

In [ ]:
test = df_with_ratios_dropped_gvkey
test = test[test.isna().any(axis=1)]
test[test['tic'] == 'AFAP']

,datadate,gvkey,tic,sic,current_ratio,quick_ratio,cash_ratio,net_working_capital,debt_ratio,debt_to_equity_ratio,...,price_to_earnings,dividend_payout_ratio,retention_ratio,gross_margin_ratio,operating_profit_margin,ebitda_margin,debt_service_coverage_ratio,interest_coverage_ratio,year,quarter
156,2000-03-31,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,84.745763,0.282486,0.717514,NaN,NaN,NaN,NaN,NaN,2000,1
157,2000-06-30,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,49.848943,0.151057,0.848943,NaN,NaN,NaN,NaN,NaN,2000,2
158,2000-09-30,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,3
159,2000-12-31,1019,AFAP,7380,1.640074,1.550835,0.116883,3.450,0.062365,0.135467,...,34.014870,0.092937,0.907063,-17.483221,-195.148610,-185.426654,-22.792833,24.408072,2000,4
160,2001-03-31,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,75.100402,0.200803,0.799197,NaN,NaN,NaN,NaN,NaN,2001,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2019-12-31,1019,AFAP,7380,1.405545,1.159588,0.031515,6.846,0.054954,0.157569,...,13.225569,0.036738,0.963262,32.806181,-60.904102,-58.519883,-27.607201,44.666667,2019,4
236,2020-03-31,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,1
237,2020-06-30,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,2
238,2020-09-30,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,3


In [ ]:
len(df_with_ratios_dropped_gvkey)

883497

Here we can see that there are still certain stocks where there is data, but many missing values. for example for ticker AFAP, we have data missing from 2003-2016. Such tickers should be dropped as well

In [ ]:
cols_df = df_with_ratios_dropped_gvkey.drop(['gvkey', 'sic',  'tic', 'year', 'quarter', 'datadate'], axis=1)
all_na_rows = cols_df[cols_df.isna().all(axis=1)] # all rows where each feature is NA
df_with_nans = df_with_ratios_dropped_gvkey.loc[all_na_rows.index]
df_with_nans_counts = df_with_nans.groupby('gvkey').size() # Number of nan rows for each gvkey
gvkey_with_bad_data = df_with_nans_counts[df_with_nans_counts > 10]

In [ ]:
print(100*len(gvkey_with_bad_data) / len(df_with_ratios_dropped_gvkey['gvkey'].unique()))

2.992529348986126


We can decide here the threshold for which we want to drop our gvkeys. We drop gvkeys with more than 10 missing rows with all NANS. This causes us to drop 2.99% of gvkeys. 

In [ ]:
df_with_ratios_dropped_gvkey_v2 = df_with_ratios_dropped_gvkey[~df_with_ratios_dropped_gvkey['gvkey'].isin(gvkey_with_bad_data.index)]
print(f"% of total rows lost in total after this step: {(len(df_with_ratios) - len(df_with_ratios_dropped_gvkey_v2)) * 100 / len(df_with_ratios)  }%")

% of total rows lost in total after this step: 21.50732333552728%


In [ ]:
nan_breakdown = pd.DataFrame()
nan_breakdown['pct_nan'] = (df_with_ratios_dropped_gvkey_v2.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown

,pct_nan
datadate,0.000000
gvkey,0.000000
tic,0.022995
sic,0.000000
current_ratio,17.390932
quick_ratio,18.483870
cash_ratio,18.525577
net_working_capital,17.353787
debt_ratio,8.907723
debt_to_equity_ratio,9.064216


Our % of rows dropped increased from 17% to 21%. We can reduce this perentage by increasing the tolerance of how many nan rows a stock can have. This is balancing data quantity vs quality. % of nans for many rows also decreased. 

In [ ]:
# dealing with inf value
inf_breakdown = pd.DataFrame()
inf_breakdown['pct_inf'] = (100 * df_with_ratios_dropped_gvkey_v2.isin([float('inf'), float('-inf')]).sum()) / len(df_with_ratios_dropped_gvkey_v2)
inf_breakdown

,pct_inf
datadate,0.000000
gvkey,0.000000
tic,0.000000
sic,0.000000
current_ratio,0.082430
quick_ratio,0.082311
cash_ratio,0.071874
net_working_capital,0.000000
debt_ratio,0.182531
debt_to_equity_ratio,0.011742


Remove INF values for a gvkey but replacing it with its largest value. Replace negative INF with its smallest value

In [ ]:
non_numerical_cols = ['gvkey', 'sic', 'tic', 'year', 'quarter', 'datadate']
numerical_cols = [col for col in df_with_ratios_dropped_gvkey_v2.columns if col not in non_numerical_cols]
df_without_inf = pd.DataFrame()
for col in non_numerical_cols: 
    df_without_inf[col] = df_with_ratios_dropped_gvkey_v2[col]
for col in numerical_cols: 
    df_without_inf[col] = df_with_ratios_dropped_gvkey_v2.groupby(['gvkey'])[col].transform(lambda x: x.replace([float('inf')], x.replace([float('inf')], float('-inf')).max())).transform(lambda x: x.replace([float('-inf')], x.replace([float('-inf')], float('inf')).min()))

In [ ]:
inf_breakdown = pd.DataFrame()
inf_breakdown['pct_inf'] = (100 * df_without_inf.isin([float('inf'), float('-inf')]).sum()) / len(df_without_inf)
inf_breakdown

,pct_inf
gvkey,0.0
sic,0.0
tic,0.0
year,0.0
quarter,0.0
datadate,0.0
current_ratio,0.0
quick_ratio,0.0
cash_ratio,0.0
net_working_capital,0.0


In [ ]:
df_without_inf.head()

,gvkey,sic,tic,year,quarter,datadate,current_ratio,quick_ratio,cash_ratio,net_working_capital,...,day_sales_outstanding,working_capital_turnover,price_to_earnings,dividend_payout_ratio,retention_ratio,gross_margin_ratio,operating_profit_margin,ebitda_margin,debt_service_coverage_ratio,interest_coverage_ratio
0,1004,5080,AIR,2000,1,2000-02-29,2.981531,1.176928,-0.021609,350.202,...,NaN,NaN,59.375000,0.212500,0.787500,18.234795,-72.491564,-70.807951,-0.958372,4.223950
1,1004,5080,AIR,2000,2,2000-05-31,3.120983,1.069792,-0.042786,347.451,...,531.051575,0.645246,154.166667,0.944444,0.055556,18.944015,-75.280235,-73.157425,-0.819497,2.287927
2,1004,5080,AIR,2000,3,2000-08-31,3.032279,0.989191,-0.003399,341.996,...,513.171702,0.701345,93.750000,0.708333,0.291667,16.176118,-77.799562,-75.858047,-0.823112,2.432365
3,1004,5080,AIR,2000,4,2000-11-30,2.718261,0.897575,0.009653,325.382,...,594.639896,0.633329,64.843750,0.531250,0.468750,18.857738,-73.583647,-71.446755,-0.674333,2.793634
4,1004,5080,AIR,2001,1,2001-02-28,3.136912,1.025058,-0.002824,358.685,...,637.845577,0.584946,68.000000,0.425000,0.575000,20.259808,-71.119752,-68.726602,-0.619269,3.174489


In [ ]:
df_without_inf['sic_2']  = df['sic'].astype(str).str[:2].astype(int) # use major SIC group

In [ ]:
sector_date_median = df_without_inf.groupby(['sic_2', 'year', 'quarter']).median()
sector_date_median

gvkey     sic  current_ratio  quick_ratio  cash_ratio  \
sic_2 year quarter                                                            
10    2000 1        26748.0  1040.0       2.027228     1.337097   -0.008772   
           2        26837.0  1040.0       2.235176     1.330361   -0.014108   
           3        26906.0  1040.0       1.885593     1.287574   -0.015638   
           4        26910.0  1040.0       1.567034     1.266471   -0.025805   
      2001 1        27309.0  1040.0       1.650396     1.168624   -0.011111   
...                     ...     ...            ...          ...         ...   
99    2022 2        37182.5  9995.0       0.343192     0.343192   -0.011795   
           3        37493.0  9995.0       0.364865     0.364865   -0.001933   
           4        37412.5  9995.0       0.298987     0.298987   -0.006556   
      2023 1        37548.0  9995.0       0.465569     0.465569   -0.011681   
           2        37543.0  9995.0       0.320375     0.320375    0.000000   

                    net_working_capital  debt_ratio  debt_to_equity_ratio  \
sic_2 year quarter                                                          
10    2000 1                     1.2575    0.034892              0.004476   
           2                     0.6945    0.035794              0.003925   
           3                     0.5640    0.032941              0.003043   
           4                     0.4460    0.048032              0.010961   
      2001 1                     0.4030    0.039936              0.005299   
...                                 ...         ...                   ...   
99    2022 2                    -0.1665    0.005729              0.000000   
           3                    -0.2290    0.062021             -0.000000   
           4                    -0.1845    0.130641              0.000000   
      2023 1                    -0.3430    0.170179              0.000000   
           2                    -0.5710    0.116739              0.000000   

                    equity_ratio  cashflow_to_debt_ratio  ...  \
sic_2 year quarter                                        ...   
10    2000 1            0.742389               -0.226882  ...   
           2            0.738709               -0.342188  ...   
           3            0.777583               -0.492892  ...   
           4            0.695705               -0.576022  ...   
      2001 1            0.710131               -0.275648  ...   
...                          ...                     ...  ...   
99    2022 2            0.570273               -0.579582  ...   
           3            0.504746               -0.739426  ...   
           4            0.292845               -0.725869  ...   
      2023 1            0.189131               -0.221521  ...   
           2            0.152877               -0.385170  ...   

                    day_sales_outstanding  working_capital_turnover  \
sic_2 year quarter                                                    
10    2000 1                   140.100516                       0.0   
           2                   197.003250                      -0.0   
           3                   226.385111                       0.0   
           4                   217.759814                      -0.0   
      2001 1                   256.164684                       0.0   
...                                   ...                       ...   
99    2022 2                   225.204959                      -0.0   
           3                   182.144371                      -0.0   
           4                   205.573019                       0.0   
      2023 1                   230.792775                      -0.0   
           2                   155.115823                      -0.0   

                    price_to_earnings  dividend_payout_ratio  retention_ratio  \
sic_2 year quarter                                                              
10    2000 1                -6.280096                   -0.0

In [ ]:
cols_to_check = [col for col in df_without_inf.columns if col not in ['datadate', 'year', 'quarter', 'sic', 'gvkey', 'tic', 'sic_2']]
cols_to_check

['current_ratio',
 'quick_ratio',
 'cash_ratio',
 'net_working_capital',
 'debt_ratio',
 'debt_to_equity_ratio',
 'equity_ratio',
 'cashflow_to_debt_ratio',
 'net_profit_margin',
 'return_on_assets',
 'asset_turnover',
 'inventory_turnover',
 'days_in_inventory',
 'receivables_turnover',
 'day_sales_outstanding',
 'working_capital_turnover',
 'price_to_earnings',
 'dividend_payout_ratio',
 'retention_ratio',
 'gross_margin_ratio',
 'operating_profit_margin',
 'ebitda_margin',
 'debt_service_coverage_ratio',
 'interest_coverage_ratio']

In [ ]:
#imputation using sic code
def fill_with_median(x):
    sic = x['sic_2']
    year = x['year']
    quarter = x['quarter']
    for col in cols_to_check: 
        if np.isnan(x[col]): 
            median = sector_date_median[sector_date_median.index == (sic,year,quarter)][col].iloc[0]
            x[col] = median
    return x

In [ ]:
#use pandarallel to speed up the process

pandarallel.initialize()
df_inputed = df_without_inf.parallel_apply(fill_with_median, axis=1)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [ ]:
nan_breakdown = pd.DataFrame()
nan_breakdown['pct_nan_before_sector_inpute'] = (df_with_ratios_dropped_gvkey_v2.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown['pct_nan_after_sector_inpute'] = (df_inputed.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown

,pct_nan_before_sector_inpute,pct_nan_after_sector_inpute
datadate,0.000000,0.000000
gvkey,0.000000,0.000000
tic,0.022995,0.022995
sic,0.000000,0.000000
current_ratio,17.390932,0.002048
quick_ratio,18.483870,0.002048
cash_ratio,18.525577,0.002234
net_working_capital,17.353787,0.002048
debt_ratio,8.907723,0.002793
debt_to_equity_ratio,9.064216,0.002793


In [ ]:
df_inputed.ffill(limit=4, inplace=True)
df_inputed.dropna(inplace=True)

In [ ]:
nan_breakdown = pd.DataFrame()
nan_breakdown['pct_nan'] = (df_inputed.isna().sum() / len(df_inputed)) * 100
nan_breakdown

,pct_nan
gvkey,0.0
sic,0.0
tic,0.0
year,0.0
quarter,0.0
datadate,0.0
current_ratio,0.0
quick_ratio,0.0
cash_ratio,0.0
net_working_capital,0.0


In [ ]:
df_inputed.to_pickle('compustat_inputed.pkl') 
compustat = pd.read_pickle('compustat_imputed.pkl')

In [ ]:
with open('../token.json', 'r') as token_file:
    token_data = json.load(token_file)
    project_id = token_data.get('project_id', 'default-project-id')

dataset_id = "capstone"
table_id = 'compustat_data'

to_gbq(df_inputed, destination_table=f'{project_id}.{dataset_id}.{table_id}', project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 996.04it/s]
